In [1]:
import pandas as pd
raw_data = pd.read_csv('speed_data_data.csv')
# assume 'data' is your dataset
missing_values = raw_data.isnull().sum()
print(missing_values)


gender       0
age         95
income    4099
goal        79
career      89
dec          0
attr       202
sinc       277
intel      296
fun        350
amb        712
shar      1067
like       240
prob       309
met        375
dtype: int64


In [8]:
data = raw_data.copy()
categorical_cols = data.select_dtypes(include=[object]).columns
data = data.dropna(subset=categorical_cols, how='all')




In [11]:
import numpy as np
from sklearn.impute import KNNImputer

numerical_cols = data.select_dtypes(include=[np.number]).columns
imputer = KNNImputer(n_neighbors=5)
data[numerical_cols] = imputer.fit_transform(data[numerical_cols])

In [12]:
data

,gender,age,income,goal,career,dec,attr,sinc,intel,fun,amb,shar,like,prob,met
0,0.0,21.0,69487.0,2.0,lawyer,1.0,6.0,9.0,7.0,7.0,6.0,5.0,7.0,6.0,2.0
1,0.0,21.0,69487.0,2.0,lawyer,1.0,7.0,8.0,7.0,8.0,5.0,6.0,7.0,5.0,1.0
2,0.0,21.0,69487.0,2.0,lawyer,1.0,5.0,8.0,9.0,8.0,5.0,7.0,7.0,6.2,1.0
3,0.0,21.0,69487.0,2.0,lawyer,1.0,7.0,6.0,8.0,7.0,6.0,8.0,7.0,6.0,2.0
4,0.0,21.0,69487.0,2.0,lawyer,1.0,5.0,6.0,7.0,7.0,6.0,6.0,6.0,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,1.0,25.0,26779.0,1.0,assistant master of the universe (otherwise it...,0.0,3.0,5.0,5.0,5.0,6.2,3.4,2.0,5.0,0.0
8374,1.0,25.0,22668.0,1.0,assistant master of the universe (otherwise it...,0.0,4.0,6.0,8.0,4.0,4.0,3.8,4.0,4.0,0.0
8375,1.0,25.0,22668.0,1.0,assistant master of the universe (otherwise it...,0.0,4.0,7.0,8.0,8.0,8.0,4.6,6.0,5.0,0.0
8376,1.0,25.0,16767.0,1.0,assistant master of the universe (otherwise it...,0.0,4.0,6.0,5.0,4.0,5.8,5.0,5.0,5.0,0.0


In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numerical_col = data.select_dtypes(include=[np.number]).columns
numerical_col_no_income = numerical_col.drop('income') #Exclde the income column
data[numerical_col_no_income] = scaler.fit_transform(data[numerical_col_no_income])



In [14]:
data = pd.get_dummies(data, columns=['gender'], drop_first=True)


In [15]:
from sklearn.model_selection import train_test_split

train_set, temp_set = train_test_split(data, test_size=0.3, random_state=42)
test_set, val_set = train_test_split(temp_set, test_size=0.5, random_state=42 )
train_set = train_set.drop('career', axis=1)
test_set = test_set.drop('career', axis=1)
train_set['dec'] = train_set['dec'].astype(int)
test_set['dec'] = test_set['dec'].astype(int)

In [16]:
from imblearn.over_sampling import RandomOverSampler

# Apply oversampling
ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(train_set.drop('dec', axis=1), train_set['dec'])

# Replace the original training set with the oversampled one
train_set = train_set.drop('dec', axis=1)
train_set['dec'] = y_res

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [21]:
# Creating Linear Regression Model
model = LogisticRegression()

# Training model on training data
train_set.dropna(subset=['dec'], inplace=True)  # Remove rows with NaN in the target variable
model.fit(train_set.drop('dec', axis=1), train_set['dec'])

# Making predictions
predictions = model.predict(test_set.drop('dec', axis=1))

# Evaluating Model's Performance
accuracy = accuracy_score(test_set['dec'], predictions)
print(f'Accuracy: {accuracy:.3f}')
print(classification_report(test_set['dec'], predictions))
print(confusion_matrix(test_set['dec'], predictions))


Accuracy: 0.433
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       705
           1       0.43      1.00      0.60       538

    accuracy                           0.43      1243
   macro avg       0.22      0.50      0.30      1243
weighted avg       0.19      0.43      0.26      1243

[[  0 705]
 [  0 538]]


C:\Users\Goodness\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Goodness\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Goodness\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
train_set['dec'].value_counts()